In [11]:
import pandas as pd
import requests
import json
pd.set_option('display.max_colwidth', None)

In [4]:
df = pd.read_excel('cadastro_grupos/Share of Voice - Grupos_TRATADO.xlsx', sheet_name='Planilha1', engine='openpyxl')

In [12]:
df.query('tipo_link == "Grupo" and id_api.isna()')

,cust_id,status_cust_id,class_atual,deals_group_on_wpp,link,tipo_link,id_api
7,1299080285,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/BQNwi2Odj1YAy2bkvl3MN0,Grupo,NaN
9,1374758175,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/CYAnpD33S3NJcPLveFprJm,Grupo,NaN
90,2221972417,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/ESRK7K8eTez9n4tsWi8Ca3?mode=wwc,Grupo,NaN
105,147294155,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/GlZbsfe4H0q7GdhETjruhL,Grupo,NaN
183,470806864,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/ED3SWc4c1ZT2oeACRALSiI?fbclid=PAZXh0bgNhZW0CMTEAAadYUujDSi-fDWl6u9ooOCnZFX8wkV_ZT9uZW8WyHrud-Tx_C2WBQxIefWv4yw_aem_CF9ezvNK1O2X6PKuhvj8oQ,Grupo,NaN
328,423827714,Confirmado,MElIpro,Sim,https://chat.whatsapp.com/JRsewVssOZsKY1OvdyAf3s,Grupo,NaN
367,2662787671,Confirmado,MElIpro,Sim,https://chat.whatsapp.com/J35VKuP8EOLFtZaq2Hu2it?fbclid=PAZXh0bgNhZW0CMTEAAac1DEub0kufl5CnzTfhWxGfhywYwtK4yAdhWoN_gyKfcNytXU5EFC1uVxkdvw_aem_mb0Wss1twKAEcCihC5X0gA,Grupo,NaN
398,268850426,Pendente,top KAM Cross/Tech,Sim,https://chat.whatsapp.com/EtM3xhWp5wcFuqkVNdLOL1?mode=ac_t&fbclid=PAZXh0bgNhZW0CMTEAAcvKmLCoRYh1JrXVQa4GIsd0EnkbBnFBQ4AcdQWmNVlGTiH5r-fPAM9WgGJDw_aem_9ZENO9O-OpjiO-ebHt4fdA,Grupo,NaN


In [ ]:
https://chat.whatsapp.com/BQNwi2Odj1YAy2bkvl3MN0	"id": "553499359585-1626705466@g.us"
https://chat.whatsapp.com/CYAnpD33S3NJcPLveFprJm	"id": "120363358188083030@g.us"
https://chat.whatsapp.com/ESRK7K8eTez9n4tsWi8Ca3
https://chat.whatsapp.com/GlZbsfe4H0q7GdhETjruhL	"id": "120363147672890533@g.us"
https://chat.whatsapp.com/ED3SWc4c1ZT2oeACRALSiI
https://chat.whatsapp.com/JRsewVssOZsKY1OvdyAf3s	"id": "120363403071832784@g.us"
https://chat.whatsapp.com/J35VKuP8EOLFtZaq2Hu2it
https://chat.whatsapp.com/EtM3xhWp5wcFuqkVNdLOL1

In [14]:
df = pd.read_json('groups.json')

In [16]:
df.to_csv('grupos_api.csv', index=False)